In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
import gensim
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier

In [2]:
import sys
sys.path.append('/home/sarahwie/Documents/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/DeepLearningMovies_datasets/'

In [12]:
nltk.download()
# Load the punkt tokenizer
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### This function replaces all the Kaggle word parsing/cleaning by importing KaggleWord2VecUtility package :

Need to use this function instead of parsing by sentence:
Note* can choose to remove stopwords.

In [19]:
def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, remove_stopwords=False ))
    return clean_reviews

In [4]:
# Read data from files
train = pd.read_csv( os.path.join(DATADIR, 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( os.path.join(DATADIR, "unlabeledTrainData.tsv"), header=0,  delimiter="\t", quoting=3 )

In [5]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews " \
     "and %d unlabeled reviews\n" % (train["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews and 50000 unlabeled reviews



In [6]:
train.ix[0,]

id                                                    "5814_8"
sentiment                                                    1
review       "With all this stuff going down at the moment ...
Name: 0, dtype: object

### Don't know where this came from. Gets an even split train/test set from our labelled training set

OH because Kaggle's test set has no labels so we need our own labelled test set.

In [7]:
fracTrain = 0.5
nSamples = train.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = train.ix[order[:splitIndex],:]
test1 = train.ix[order[splitIndex:],:]

In [8]:
print train1.shape
print test1.shape

(12500, 3)
(12500, 3)


# Word2Vec inversion

Here is some stuff to split the labelled training set into positive and negative reviews. NOTE: takes a long time to run.

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into words and appends to a list of positive and negative words, respectively. Also does the same for the unlabelled training reviews and their words.

In [20]:
# ****** Split the labeled and unlabeled training sets into clean words
#
inxs_pos = np.where(train1["sentiment"] == 1)[0].tolist()
inxs_neg = np.where(train1["sentiment"] == 0)[0].tolist()

print "Parsing words from training set"
reviews_pos = train1.iloc[inxs_pos]
words_pos = getCleanReviews(reviews_pos)

reviews_neg = train1.iloc[inxs_neg]
words_neg = getCleanReviews(reviews_neg)   
    
print "Parsing words from unlabeled set"
words_unlabelled = getCleanReviews(unlabeled_train)

Parsing words from training set
Parsing words from unlabeled set


Now do the same for all sentences without splitting to build a vocabulary:

In [21]:
# ****** Split the labeled and unlabeled training sets into clean words, don't separate by class
#
words = [] #initialize an empty list

print "Parsing words from training set"
words += getCleanReviews(train1)

print "Parsing words from unlabeled set"
words += getCleanReviews(unlabeled_train)

Parsing words from training set
Parsing words from unlabeled set


In [25]:
#print words[0:2]

In [26]:
#print words_neg[0:1]

In [27]:
#print words_pos[0:1]

Some stuff from Taddy's code (much simpler than Kaggle method). 

In [28]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [29]:
basemodel.build_vocab(words) 

Train 2 models off of the base model (one pos & one neg). This changes now that we have words only.

In [37]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(words_pos, total_examples=len(words_pos) )
models[1].train(words_neg, total_examples=len(words_neg) )

3222425

In [61]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of words
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.

***HERE is where I need to modify how the class probabilities are calculated. Must average across words now instead of sentences.
"""

import pandas as pd # for quick summing within doc

def docprobwords(docs,mods):
    sentlist = [s for d in docs for s in d]
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    lhd = np.exp(llhd - llhd.max(axis=0))
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

### Test set example
As an example, we apply the inversion on the full test set.

In [40]:
# read in the test set as a list of lists of words
docs = getCleanReviews(test1)

In [43]:
print docs[0:2]

[[u'i', u'only', u'lasted', u'mins', u'before', u'self', u'preservation', u'jerked', u'me', u'out', u'of', u'the', u'empty', u'eyed', u'drooling', u'stupor', u'that', u'this', u'film', u'effortlessly', u'induced', u'and', u'propelled', u'me', u'screaming', u'back', u'to', u'the', u'video', u'shop', u'armed', u'for', u'bear', u'to', u'say', u'the', u'film', u'was', u'bad', u'would', u'be', u'a', u'missed', u'opportunity', u'to', u'use', u'words', u'interspersed', u'with', u'characters', u'from', u'the', u'top', u'keys', u'on', u'my', u'keyboard', u'just', u'to', u'keep', u'these', u'comments', u'clean', u'one', u'to', u'be', u'avoided'], [u'the', u'michael', u'keaton', u'kiddie', u'comedy', u'of', u'the', u'same', u'title', u'was', u'roundly', u'condemned', u'for', u'it', u's', u'um', u'shoddy', u'special', u'effects', u'but', u'compared', u'to', u'what', u'screaming', u'mad', u'george', u'cooked', u'up', u'for', u'this', u'horror', u'comedy', u'they', u're', u'positively', u'mind', u'b

In [64]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs[0:2], models)

In [60]:
probswords

,0,1
doc,,
0,0.661443,0.338557
1,0.673901,0.326099


In [ ]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs, models)

A numpy array of ones for the number of docs that we're predicting:

In [65]:
predswords = np.ones((probswords.shape[0]))

In [66]:
print np.shape(predswords)

(12500,)


In [67]:
predswords[np.where(probswords.iloc[:,1] > 0.5)] = 0 # The second column is actually the negative model

In [68]:
predswords

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [69]:
print predswords.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [71]:
print np.size(np.where(predswords == test1["sentiment"]))*1./np.size(predswords)

0.4892
